In [75]:
# pip install transformers datasets
import torch
print("torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=None,
    device=0        # use GPU 0
)
tokenizer = AutoTokenizer.from_pretrained("j-hartmann/emotion-english-distilroberta-base")


torch: 2.5.1+cu121
CUDA available: True
GPU: NVIDIA GeForce RTX 3060 Laptop GPU


Device set to use cuda:0


In [76]:
import os
import json

# Dictionary to hold all loaded texts
texts = {}

for filename in os.listdir("../data"):
    if filename.endswith(".txt"):
        with open("../data/" + filename, "r", encoding="utf-8") as f:
            data = json.load(f)
            
            # Use filename without extension as key
            key_base = os.path.splitext(filename)[0]
                
            # Save all keys from the JSON file (like 'text', 'reference_text') inside a sub-dictionary
            texts[key_base] = data.copy()

# Example access
print(texts["drones_of_deception_temple"]["text"])
print(texts["cyber_attack_killeen_power_plant"]["reference_text"])

As tensions between Donovia and the Western Bloc reach a boiling point, reports of mysterious drone sightings in Temple, Texas, have left residents on edge and sparked concerns about foreign interference in domestic affairs. Central Texas City Grapples with Mysterious Sightings Amidst Growing Tensions The Donovian government has issued a statement denying any involvement, but insiders claim otherwise. The situation is further complicated by the escalating global power struggle between Donovia and its Western adversaries. Temple, Texas, a city in Central Texas with a population of over 70,000 residents, has become an unlikely focal point for international attention. The mysterious drone sightings have raised concerns about national security and the potential for foreign interference in domestic affairs. In recent weeks, numerous reports have surfaced from Temple residents claiming to have seen unidentified drones hovering above their homes or neighborhoods. The incidents have been chara

In [77]:
# The directory where your expected output files are stored
expected_outputs_dir = "../data/expected_outputs"

# A dictionary to hold the expected results
expected_results = {}

# Check if the directory exists
if os.path.isdir(expected_outputs_dir):
    # Loop through each file in the directory
    for filename in os.listdir(expected_outputs_dir):
        # Process only JSON files
        if filename.endswith(".json"):
            file_path = os.path.join(expected_outputs_dir, filename)
            # Use the filename without the .json extension as the key
            key = os.path.splitext(filename)[0]
            with open(file_path, 'r', encoding='utf-8') as f:
                # Load the JSON data and add it to the dictionary
                expected_results[key] = json.load(f)

# Print the resulting dictionary (optional, for verification)
print("Loaded Expected Results:")
print(expected_results["drones_of_deception_temple"])

Loaded Expected Results:
{'fear': 3, 'stress': 3, 'morale': 1, 'trust_in_authorities': 1}


In [78]:
from collections import defaultdict


def chunk_text(text, tokenizer, max_tokens=400):
    tokens = tokenizer.encode(text)
    chunks = [tokens[i:i + max_tokens] for i in range(0, len(tokens), max_tokens)]
    return [tokenizer.decode(chunk) for chunk in chunks]


def avg_emotion_scores(text, classifier, tokenizer):
    if (len(tokenizer.encode(text)) <= 400):
        result = classifier(text)[0]
        return {item['label']: item['score'] for item in result}

    chunks = chunk_text(text, tokenizer)
    cumulative_scores = defaultdict(float)
    total_chars = 0

    for chunk in chunks:
        result = classifier(chunk)[0]
        chunk_len = len(chunk)
        total_chars += chunk_len
        for item in result:
            cumulative_scores[item['label']] += item['score'] * chunk_len

    averaged_scores = {label: score / total_chars for label, score in cumulative_scores.items()}
    return averaged_scores

In [79]:
drones_of_deception_temple_results = avg_emotion_scores(texts["drones_of_deception_temple"]["text"], classifier, tokenizer)
print("drones_of_deception_temple: ", drones_of_deception_temple_results)

cyber_attack_killeen_power_plant_results = avg_emotion_scores(texts["cyber_attack_killeen_power_plant"]["reference_text"], classifier, tokenizer)
print("cyber_attack_killeen_power_plant: ", cyber_attack_killeen_power_plant_results)

food_shortage_rumors_results = avg_emotion_scores(texts["food_shortage_rumors"]["reference_text"], classifier, tokenizer)
print("food_shortage_rumors: ", food_shortage_rumors_results)

nato_exercises_boost_confidence_results = avg_emotion_scores(texts["nato_exercises_boost_confidence"]["reference_text"], classifier, tokenizer)
print("nato_exercises_boost_confidence: ", nato_exercises_boost_confidence_results)

virus_panic_harker_heights_results = avg_emotion_scores(texts["virus_panic_harker_heights"]["text"], classifier, tokenizer)
print("virus_panic_harker_heights: ", virus_panic_harker_heights_results)

factory_explosion_false_flag_results = avg_emotion_scores(texts["factory_explosion_false_flag"]["reference_text"], classifier, tokenizer)
print("factory_explosion_false_flag: ", factory_explosion_false_flag_results)

community_safety_initiative_results = avg_emotion_scores(texts["community_safety_initiative"]["text"], classifier, tokenizer)
print("community_safety_initiative: ", community_safety_initiative_results)


Token indices sequence length is longer than the specified maximum sequence length for this model (1129 > 512). Running this sequence through the model will result in indexing errors


drones_of_deception_temple:  {'fear': 0.7602706353398871, 'anger': 0.10429053363751974, 'neutral': 0.1027084575187392, 'disgust': 0.014090057810052512, 'sadness': 0.010059505021061598, 'surprise': 0.006131273001968546, 'joy': 0.0024495590180118303}
cyber_attack_killeen_power_plant:  {'fear': 0.5403466023699296, 'neutral': 0.313806427239273, 'surprise': 0.05438798490393844, 'sadness': 0.042338932968801986, 'anger': 0.0365296998182841, 'disgust': 0.009242527178286957, 'joy': 0.0033478991274425787}
food_shortage_rumors:  {'disgust': 0.43370577692985535, 'anger': 0.2831280827522278, 'neutral': 0.11049191653728485, 'surprise': 0.07179553806781769, 'sadness': 0.06034587323665619, 'fear': 0.03899148479104042, 'joy': 0.0015412700595334172}
nato_exercises_boost_confidence:  {'fear': 0.4742882251739502, 'joy': 0.2648555338382721, 'neutral': 0.21620242297649384, 'anger': 0.020228777080774307, 'disgust': 0.011500043794512749, 'sadness': 0.006775944493710995, 'surprise': 0.006148991174995899}
virus

In [80]:
from typing import Dict

# Clamps emotions to [-1, 1]
def clamp(emotions: Dict[str, float]) -> Dict[str, float]:
    for emotion in emotions:
        if emotions[emotion] < -1:
            emotions[emotion] = -1
        elif emotions[emotion] > 1:
            emotions[emotion] = 1
    return emotions

# Map emotions to fear, stress, trust and morale
def emotions_to_fsmt(emotions: Dict[str, float]) -> Dict[str, int]:
    fear = emotions.get("fear")
    anger = emotions.get("anger")
    neutral = emotions.get("neutral")
    disgust = emotions.get("disgust")
    sadness = emotions.get("sadness")
    joy = emotions.get("joy")

    fsmt_dict = {}
    
    if(fear > anger + neutral + disgust + sadness + joy or fear > 0.8):
        fsmt_dict["Fear Level"] = 3
    elif(fear > 0.5 or fear > joy + neutral):
        fsmt_dict["Fear Level"] = 2
    else:
        fsmt_dict["Fear Level"] = 1

    if(sadness + anger > joy + neutral or fsmt_dict["Fear Level"] == 3):
        fsmt_dict["Stress Level"] = 3
    elif(sadness + anger > 0.5 or fsmt_dict["Fear Level"] == 2):
        fsmt_dict["Stress Level"] = 2
    else:
        fsmt_dict["Stress Level"] = 1

    if(sadness + disgust + fear < neutral + joy):
        fsmt_dict["Morale Level"] = 3
    elif(joy + neutral > 0.5):
        fsmt_dict["Morale Level"] = 2
    else:
        fsmt_dict["Morale Level"] = 1

    if(sadness + disgust + fear < joy):
        fsmt_dict["Trust Level"] = 3
    elif(joy + neutral > sadness + disgust + fear):
        fsmt_dict["Trust Level"] = 2
    else:
        fsmt_dict["Trust Level"] = 1

    return fsmt_dict

In [81]:

print('drones_of_deception_temple',emotions_to_fsmt(drones_of_deception_temple_results))
print("Expected: ", expected_results["drones_of_deception_temple"])
print("\n")

print('cyber_attack_killeen_power_plant',emotions_to_fsmt(cyber_attack_killeen_power_plant_results))
print("Expected: ", expected_results["cyber_attack_killeen_power_plant"])
print("\n")

print('food_shortage_rumors',emotions_to_fsmt(food_shortage_rumors_results))
print("Expected: ", expected_results["food_shortage_rumors"])
print("\n")

print('nato_exercises_boost_confidence',emotions_to_fsmt(nato_exercises_boost_confidence_results))
print("Expected: ", expected_results["nato_exercises_boost_confidence"])
print("\n")

print('virus_panic_harker_heights',emotions_to_fsmt(virus_panic_harker_heights_results))
print("Expected: ", expected_results["virus_panic_harker_heights"])
print("\n")

print('factory_explosion_false_flag',emotions_to_fsmt(factory_explosion_false_flag_results))
print("Expected: ", expected_results["factory_explosion_false_flag"])
print("\n")

print('community_safety_initiative',emotions_to_fsmt(community_safety_initiative_results))
print("Expected: ", expected_results["community_safety_initiative"])
print("\n")

drones_of_deception_temple {'Fear Level': 3, 'Stress Level': 3, 'Morale Level': 1, 'Trust Level': 1}
Expected:  {'fear': 3, 'stress': 3, 'morale': 1, 'trust_in_authorities': 1}


cyber_attack_killeen_power_plant {'Fear Level': 3, 'Stress Level': 3, 'Morale Level': 1, 'Trust Level': 1}
Expected:  {'fear': 3, 'stress': 3, 'morale': 1, 'trust_in_authorities': 2}


food_shortage_rumors {'Fear Level': 1, 'Stress Level': 3, 'Morale Level': 1, 'Trust Level': 1}
Expected:  {'fear': 3, 'stress': 3, 'morale': 1, 'trust_in_authorities': 1}


nato_exercises_boost_confidence {'Fear Level': 1, 'Stress Level': 1, 'Morale Level': 1, 'Trust Level': 1}
Expected:  {'fear': 1, 'stress': 1, 'morale': 3, 'trust_in_authorities': 3}


virus_panic_harker_heights {'Fear Level': 3, 'Stress Level': 3, 'Morale Level': 1, 'Trust Level': 1}
Expected:  {'fear': 3, 'stress': 3, 'morale': 1, 'trust_in_authorities': 1}


factory_explosion_false_flag {'Fear Level': 1, 'Stress Level': 1, 'Morale Level': 1, 'Trust Level': 